In [3]:
from mp_api.client import MPRester
import pandas as pd
import matplotlib.pyplot as plt

API_KEY = "predX42d0PDrOGcfMdmTpLTtvij6k0Ts"

def find_solar_cell_candidates(min_gap=1.1, max_gap=1.7):
    """Find materials with ideal band gaps and rank by stability (energy above hull)."""
    with MPRester(API_KEY) as mpr:
        results = mpr.summary.search(
            band_gap=(min_gap, max_gap),
            fields=["material_id", "formula_pretty", "band_gap", "energy_above_hull"]
        )

    data = [
        {
            "material_id": doc.material_id,
            "formula": doc.formula_pretty,
            "band_gap": doc.band_gap,
            "energy_above_hull": doc.energy_above_hull
        }
        for doc in results
        if doc.energy_above_hull is not None
    ]

    df = pd.DataFrame(data)
    df = df.sort_values(by="energy_above_hull")  # Lower is more stable
    return df


def plot_energy_vs_band_gap(df):
    """Visualize band gap vs energy above hull."""
    plt.figure(figsize=(8, 5))
    plt.scatter(df["band_gap"], df["energy_above_hull"], alpha=0.7, color="green", edgecolor="black")
    plt.title("Stability vs. Band Gap (Ideal Solar Cell Range)")
    plt.xlabel("Band Gap (eV)")
    plt.ylabel("Energy Above Hull (eV)")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def main():
    df = find_solar_cell_candidates()
    
    if df.empty:
        print("No materials found in the ideal solar cell band gap range.")
        return

    print("Top 10 most stable materials (lowest energy above hull):\n")
    print(df[["formula", "band_gap", "energy_above_hull"]].head(10).to_string(index=False))

    plot = input("\nDo you want to plot energy vs band gap? (y/n): ").strip().lower()
    if plot == "y":
        plot_energy_vs_band_gap(df)

if __name__ == "__main__":
    main()


C:\Users\tadic\AppData\Local\Temp\ipykernel_19928\3482468466.py:10: DeprecationWarning: Accessing summary data through MPRester.summary is deprecated. Please use MPRester.materials.summary instead.
  results = mpr.summary.search(
Retrieving SummaryDoc documents: 100%|██████████████████████████████████████████████████████| 11183/11183 [00:03<00:00, 2894.19it/s]


Top 10 most stable materials (lowest energy above hull):

   formula  band_gap  energy_above_hull
    AgHgSI    1.3584                0.0
  AgAsPbS3    1.3860                0.0
   ZrN6O17    1.5707                0.0
      ZrIN    1.2172                0.0
   Ag3AsS3    1.6278                0.0
   Ag5BiO4    1.5002                0.0
    Ag2SO4    1.2289                0.0
   Ag2TeS3    1.2999                0.0
Na5SrNbAs4    1.1673                0.0
  Zr6CCl14    1.2812                0.0



Do you want to plot energy vs band gap? (y/n):  n
